This notebook demonstrates how to use Quest to download imagery from a Web Map Tile Service (WMTS).

In addition to quest the following packages need to be installed to use this notebook:
 * holoviews
 * geoviews
 * param
 * paramnb
 * xarray
 
The can be installed with the following command:
```
conda install -c conda-forge -c pyviz/label/dev holoviews geoviews param paramnb xarray
```

In [ ]:
import quest
import param
import xarray as xr
import geoviews as gv
import holoviews as hv
from holoviews.streams import BoxEdit
from cartopy import crs as ccrs
from paramnb import Widgets

hv.extension('bokeh')

In [ ]:
quest_service = 'svc://wmts:seamless_imagery'
tile_service_options = quest.api.download_options(quest_service, fmt='param')[quest_service]
tile_service_options.params()['bbox'].precedence = -1  # hide bbox input
Widgets(tile_service_options)

In [ ]:
tiles = gv.WMTS(tile_service_options.url, extents=(-180, -80, 180, 80), 
                crs=ccrs.PlateCarree()).options(width=950, height=600)
boxes = gv.Polygons([], crs=ccrs.PlateCarree()).options(fill_alpha=0.4, color='blue', 
                                                        line_color='blue', line_width=2)
box_stream = BoxEdit(source=boxes)
tiles * boxes

In [ ]:
if box_stream.element:
    xs, ys = box_stream.element.array().T
    bbox = list(gv.util.project_extents((xs[0], ys[0], xs[2], ys[1]), ccrs.GOOGLE_MERCATOR, ccrs.PlateCarree()))
else:
    bbox = [-72.43925984610391, 45.8471360126193, -68.81252476472281, 47.856449699679516]
tile_service_options.bbox = bbox
print(bbox)

In [ ]:
basemap_features = quest.api.get_features(quest_service)
collection_name = 'examples'
if collection_name in quest.api.get_collections():
    pass
else:
    quest.api.new_collection(collection_name)

collection_feature = quest.api.add_features(collection_name, basemap_features[0])
options = dict(tile_service_options.param.get_param_values())
staged_id = quest.api.stage_for_download(collection_feature, options=options)[0]
quest.api.download_datasets(staged_id, raise_on_error=True)
meta = quest.api.get_metadata(staged_id)[staged_id]
meta

In [ ]:
path = meta['file_path']
arr = xr.open_rasterio(path)
image = gv.RGB((arr.x, arr.y, arr[0].values, 
        arr[1].values, arr[2].values), 
       vdims=['R', 'G', 'B']).options(width=950, height=600)
gv.tile_sources.Wikipedia * image